#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [5]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [6]:
embeddings=OllamaEmbeddings(model="gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

In [7]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [9]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [10]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='9dd27c91-fd91-4383-ae61-178e67f448f1', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(0.6126377)),
 (Document(id='d687de93-69af-40d1-8e7d-20665a3c0f4c', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we sh

In [11]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.0067667863,
 -0.027598673,
 0.00020106876,
 0.018739456,
 0.008986527,
 0.007329074,
 -0.009414776,
 -0.0015799119,
 -0.0031227258,
 -0.008183086,
 0.017613765,
 -0.00054518064,
 -0.01422918,
 0.010127306,
 0.0037902752,
 -0.010942358,
 0.043896474,
 0.021758985,
 0.017249998,
 0.013113568,
 0.003579286,
 -0.005712275,
 0.0032347278,
 0.017837567,
 -0.00061950635,
 -0.008311427,
 -0.017122636,
 -0.01939021,
 -0.006315537,
 -0.022644999,
 -0.005662139,
 -0.015513141,
 0.018343149,
 -0.013349923,
 -0.0028570618,
 -0.005916323,
 0.0230713,
 0.0066267555,
 0.0010473604,
 -0.004711334,
 0.0043376414,
 0.012091518,
 0.013877634,
 -0.016959,
 -0.014471294,
 0.0049147913,
 0.0024774652,
 0.0165385,
 -0.009533774,
 0.016076265,
 -0.1898307,
 -0.23144132,
 -0.006195538,
 0.004733946,
 0.012631372,
 -0.004633589,
 -0.019877052,
 0.008619382,
 0.006266417,
 -0.0054853815,
 0.0028427881,
 -0.016165122,
 0.00059943885,
 -0.015586294,
 -0.007604185,
 0.0015137066,
 -0.008266107,
 -0.0068563097,
 -

In [12]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='9dd27c91-fd91-4383-ae61-178e67f448f1', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='d687de93-69af-40d1-8e7d-20665a3c0f4c', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it tow

In [13]:
### Saving And Loading
db.save_local("faiss_index")

In [14]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [15]:
docs

[Document(id='9dd27c91-fd91-4383-ae61-178e67f448f1', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='d687de93-69af-40d1-8e7d-20665a3c0f4c', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it tow